In [1]:
# !wget http://downloads.tatoeba.org/exports/sentences.tar.bz2
# !bunzip2 sentences.tar.bz2
# !tar xvf sentences.tar

In [2]:
import pandas as pd

df = pd.read_csv('sentences.csv', sep = '\t')
df.head()

,1,cmn,我們試試看！
0,2,cmn,我该去睡觉了。
1,3,cmn,你在干什麼啊？
2,4,cmn,這是什麼啊？
3,5,cmn,今天是６月１８号，也是Muiriel的生日！
4,6,cmn,生日快乐，Muiriel！


In [3]:
eng = df.loc[df['cmn'] == 'eng']

In [4]:
eng.shape

(1289172, 3)

In [5]:
eng = eng.sample(n = 500000)

In [6]:
eng = eng.iloc[:,-1].tolist()

In [7]:
import re
from unidecode import unidecode
import cleaning
from tqdm import tqdm

def preprocessing(string):
    string = unidecode(string)
    string = re.sub(
        'http\S+|www.\S+',
        '',
        ' '.join(
            [i for i in string.split() if i.find('#') < 0 and i.find('@') < 0]
        ),
    )
    
    chars = ',.()!:\'"/;=-'
    for c in chars:
        string = string.replace(c, f' {c} ')
        
    string = re.sub('[^A-Za-z ]+', ' ', unidecode(string))
    string = re.sub(r'[ ]+', ' ', string).strip()
    
    return string.lower()

def loop(strings):
    for i in tqdm(range(len(strings))):
        strings[i] = preprocessing(strings[i])
    return strings

In [8]:
eng = cleaning.multiprocessing(eng, loop)

100%|██████████| 31250/31250 [00:00<00:00, 48995.23it/s]


In [10]:
eng[:10]

['can you watch my stuff for a minute',
 'are you dating her',
 'why did she go there without me',
 'how do i get permission to reprint an article',
 'sami bought a jar of gherkins',
 'mary wanted to know everything about tom',
 'can we make a run for it',
 'is this where tom works',
 'he answered my questions by the exercise of his excellent memory',
 'they cry out against the new tax']

In [9]:
import json

with open('eng.json', 'w') as fopen:
    json.dump(eng, fopen)